In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# include files and datasets
From add input tab, add
Disease prediction using machine learning ->https://www.kaggle.com/datasets/kaushil268/disease-prediction-using-machine-learning

disease symptom description -> https://www.kaggle.com/datasets/itachi9604/disease-symptom-description-dataset

disease predictor -> https://www.kaggle.com/datasets/ahmed321abozeid/disease-predictor

LR-pipline ->  https://www.kaggle.com/datasets/ahmed321abozeid/lr-pipeline

symptom extractor -> https://www.kaggle.com/datasets/ahmed321abozeid/symptom-extractor

datasets to your notebook before running the code

In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/disease-predictor/Disease_Predictor.py
/kaggle/input/lr-pipeline/LR_model_pipeline.pkl
/kaggle/input/symptom-extractor/Symptoms_Extractor.py
/kaggle/input/disease-prediction-using-machine-learning/Training.csv
/kaggle/input/disease-prediction-using-machine-learning/Testing.csv
/kaggle/input/disease-symptom-description-dataset/symptom_Description.csv
/kaggle/input/disease-symptom-description-dataset/Symptom-severity.csv
/kaggle/input/disease-symptom-description-dataset/symptom_precaution.csv
/kaggle/input/disease-symptom-description-dataset/dataset.csv


In [3]:
# installing sentence transformers for the symptoms vector extraction file
%pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Load the pipeline from the file
# i will pipeline logistic regression from `models.ipnyb` file and use it
import joblib

loaded_LR_pipeline = joblib.load('/kaggle/input/lr-pipeline/LR_model_pipeline.pkl')

In [5]:
# importing functions from their files
import sys


sys.path.append('/kaggle/input/symptom-extractor')
from Symptoms_Extractor import analyze_patient_text

sys.path.append('/kaggle/input/disease-predictor')
from Disease_Predictor import disease_predictor

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/wordnet2022.zip.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# Solving wordnet Look up error 
! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # temp fix for lookup error.

In [7]:
# reading and processing describtion/precautions datasets
df_prec = pd.read_csv('/kaggle/input/disease-symptom-description-dataset/symptom_precaution.csv')
df_desc = pd.read_csv('/kaggle/input/disease-symptom-description-dataset/symptom_Description.csv')

# describtions processing: we need to set this disease's name as in the training data is 'Dimorphic hemmorhoids(piles)' and in description data is 'Dimorphic hemorhoids(piles)' only on m in hemmorhoids so it needs to be handled to avoid issues in printing the description 
df_desc.iloc[16,0] = 'Dimorphic hemmorhoids(piles)'

df_desc.set_index('Disease',inplace = True)

# precations processing
df_prec.replace(float('nan'),'No Precaution',inplace = True)

# Because diabetes in Disease column is saved as 'Diabetes ' , and this will make a lot of trouble after if not handled here
df_prec['Disease'] = df_prec['Disease'].astype(str).str.strip()

df_prec.set_index('Disease',inplace = True)

In [8]:
# symptoms vectr extraction stage


symptoms = symptoms = [
    "Itching", "Skin rash", "Nodal skin eruptions", "Continuous sneezing", "Shivering", 
    "Chills", "Joint pain", "Stomach pain", "Acidity", "Ulcers on tongue", 
    "Muscle wasting", "Vomiting", "Burning sensation during urination", "Spotting in urination", 
    "Fatigue", "Weight gain", "Anxiety", "Cold hands and feet", "Mood swings", 
    "Weight loss", "Restlessness", "Lethargy", "Patches in throat", "Irregular sugar levels", 
    "Cough", "High fever", "Sunken eyes", "Breathlessness", "Sweating", 
    "Dehydration", "Indigestion", "Headache", "Yellowish skin", "Dark urine", 
    "Nausea", "Loss of appetite", "Pain behind the eyes", "Back pain", 
    "Constipation", "Abdominal pain", "Diarrhea", "Mild fever", "Yellow urine", 
    "Yellowing of eyes", "Acute liver failure", "Fluid overload", "Swelling of stomach", 
    "Swollen lymph nodes", "Malaise", "Blurred and distorted vision", "Phlegm", 
    "Throat irritation", "Redness of eyes", "Sinus pressure", "Runny nose", 
    "Congestion", "Chest pain", "Weakness in limbs", "Fast heart rate", 
    "Pain during bowel movements", "Pain in anal region", "Bloody stool", "Irritation in anus", 
    "Neck pain", "Dizziness", "Cramps", "Bruising", "Obesity", "Swollen legs", 
    "Swollen blood vessels", "Puffy face and eyes", "Enlarged thyroid", "Brittle nails", 
    "Swollen extremities", "Excessive hunger", "Extra-marital contacts", "Drying and tingling lips", 
    "Slurred speech", "Knee pain", "Hip joint pain", "Muscle weakness", "Stiff neck", 
    "Swelling joints", "Movement stiffness", "Spinning movements", "Loss of balance", 
    "Unsteadiness", "Weakness of one side of the body", "Loss of smell", "Bladder discomfort", 
    "Foul smell of urine", "Continuous feeling of needing to urinate", "Passage of gases", 
    "Internal itching", "Toxic look (typhoid-like symptoms)", "Depression", "Irritability", 
    "Muscle pain", "Altered sensorium", "Red spots over the body", "Belly pain", 
    "Abnormal menstruation", "Discolored patches on the skin", "Watering from the eyes", 
    "Increased appetite", "Polyuria (frequent urination)", "Family history (of disease)", 
    "Mucoid sputum", "Rusty sputum", "Lack of concentration", "Visual disturbances", 
    "Receiving blood transfusion", "Receiving unsterile injections", "Coma", 
    "Stomach bleeding", "Distention of abdomen", "History of alcohol consumption", 
    "Fluid overload", "Blood in sputum", "Prominent veins on calf", "Palpitations", 
    "Painful walking", "Pus-filled pimples", "Blackheads", "Scarring", "Skin peeling", 
    "Silver-like dusting (skin condition)", "Small dents in nails", "Inflammatory nails", 
    "Blisters", "Red sores around nose", "Yellow crust ooze"
]

patient_text = input('Type what you are experiencing of pain:\n')

# Call the function
result = analyze_patient_text(patient_text, symptoms)
symptom_vector = result['presence_vector']
symptom_arr = np.array(symptom_vector).reshape(1,-1)

Type what you are experiencing of pain:
 I have continous sneezing, chills , coughing and high fever


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
disease_predictor(symptom_arr,loaded_LR_pipeline,df_desc,df_prec)

Based on symptoms you provided, You might be experiencing:
1. Allergy Disease with a chance of: 0.4914
2. Bronchial Asthma Disease with a chance of: 0.0529
3. AIDS Disease with a chance of: 0.0396
4. Common Cold Disease with a chance of: 0.0303

Your Condition highly matches: Allergy Disease, 

If you want description or precautions to be taken for Allergy Type "describtion" or "precautions", else type "Exit"


 precautions



Disease: Allergy
 precaution number 1: apply calamine
 precaution number 2: cover area with bandage
 precaution number 3: No Precaution
 precaution number 4: use ice to compress itching

Want a describtion for Allergy?


 exit
